In [1]:
!rm -r sample_data

In [2]:
!git clone https://github.com/MLinApp-FP01-Team7-24/OurProject

Cloning into 'OurProject'...
remote: Enumerating objects: 259, done.
remote: Counting objects: 100% (259/259), done.
remote: Compressing objects: 100% (234/234), done.
remote: Total 259 (delta 55), reused 16 (delta 1), pack-reused 0
Receiving objects: 100% (259/259), 18.70 MiB | 4.90 MiB/s, done.
Resolving deltas: 100% (55/55), done.


# USAD

In [3]:
%cd "OurProject/Models/usad"

/content/OurProject/Models/usad


In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import os
import argparse
import torch.utils.data as data_utils
import sklearn.metrics
from sklearn import preprocessing
from utils import *
from usad import *

In [5]:
!nvidia-smi -L
device = get_default_device()

GPU 0: Tesla T4 (UUID: GPU-a74290df-d5be-181a-9cb8-23f61c574a5c)


In [17]:
w_size = 660
z_size = 600

In [6]:
model = UsadModel(w_size, z_size)
model = to_device(model,device)

checkpoint = torch.load("model.pth")

model.encoder.load_state_dict(checkpoint['encoder'])
model.decoder1.load_state_dict(checkpoint['decoder1'])
model.decoder2.load_state_dict(checkpoint['decoder2'])

<All keys matched successfully>

In [8]:
def get_df(filepaths_csv, drop_time = False, delimiter=";"):
    # Load dataframes
    print("Loading data.")
    # Make dataframes

    dfs = [pd.read_csv(filepath_csv, sep=delimiter) for filepath_csv in filepaths_csv]
    df = pd.concat(dfs)

    # Sort columns by name !!!
    df = df.sort_index(axis=1)

    # Gestisci timestamp
    if drop_time:
        df.drop(["time"], axis=1, inplace=True)
    else:
      df['time'] = pd.to_datetime(df['time'])

    # Drop useless columns
    columns_to_drop = [column for column in df.columns if "Abb" in column or "Temperature" in column]
    df.drop(["machine_nameKuka Robot_export_active_energy",
             "machine_nameKuka Robot_import_reactive_energy"] + columns_to_drop, axis=1, inplace=True)

    # Drop NaN
    df = df.dropna(axis=0)

    print("Loading data done.\n")

    return df

In [11]:
anomaly_path = "/content/OurProject/kuka_dataset/collisions"
period = 0.1

window_len = 12
BATCH_SIZE = 128


In [12]:
filepath_csv = [os.path.join(anomaly_path, f"rec{r}_20220811_rbtc_{period}s.csv") for r in [1, 5]]

In [15]:
#Read data
df = get_df(filepath_csv, drop_time = False)
attack = df.drop(columns=['time'], axis=1)

#Scaling
min_max_scaler = preprocessing.MinMaxScaler()
x = attack.values
x_scaled = min_max_scaler.fit_transform(x)
attack = pd.DataFrame(x_scaled)

#Windowing
windows_attack = np.array([attack[i:i + window_len] for i in range(len(attack) - window_len + 1)])
windows_attack.shape

Loading data.
Loading data done.



(30764, 12, 55)

In [18]:
test_loader = torch.utils.data.DataLoader(data_utils.TensorDataset(
    torch.from_numpy(windows_attack).float().view(([windows_attack.shape[0],w_size]))
) , batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

In [ ]:
accuracy = sklearn.metrics.accuracy_score(label_eval, y_pred_eval)
accuracy

In [ ]:
f1 = sklearn.metrics.f1_score(label_eval, y_pred_eval)
f1

In [ ]:
fpr, tpr, thresholds = sklearn.metrics.roc_curve(label_eval, y_score_eval)
roc_auc = sklearn.metrics.auc(fpr, tpr)

plt.plot(fpr, tpr, label='ROC curve (area = %0.4f)' % roc_auc)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend()
plt.show()

roc_auc

In [ ]:
precison, recall, thresholds = sklearn.metrics.precision_recall_curve(label_eval, y_score_eval)
prc_auc = sklearn.metrics.auc(recall, precison)

plt.plot(recall, precison, label='PRC curve (area = %0.4f)' % prc_auc)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision Recall Curve')
plt.legend()
plt.show()

prc_auc